In [1]:
import os
os.chdir("W:/RM/src")

In [2]:
from mmdetection_rm.work.work_resource import WorkResourceFactory
factory = WorkResourceFactory()
db = factory.resource_db

***violating paths***


In [3]:
view = factory.view
view.table

,id,0
0,1,test_1
1,2,test_2
2,3,test_3
3,4,test_4
4,0,test


In [8]:
db.create("test")


🆕 새로운 ID 생성: 4


AttributeError: 'str' object has no attribute 'as_posix'

In [7]:
work = db.get(2)

In [8]:
work.task_db

***violating paths***


TaskDB(dir_db=DirDB(dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2'), factory=<rm.dirtree.factory.DirTreeFactory object at 0x000001B8C21F8940>), factory=TaskResourceFactory(dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2'), CONFIG_MANAGER_CLASS=<class 'mmdetection_rm.work.task.TaskConfigManager'>, RECORD_CLASS=<class 'mmdetection_rm.work.task.TaskRecord'>, DB_CLASS=<class 'mmdetection_rm.work.task.TaskDB'>, VIEW_CLASS=<class 'mmdetection_rm.work.task.TaskDBView'>, CONFIG_NAME='task_config'), RECORD_CLASS=<class 'mmdetection_rm.work.task.TaskRecord'>)

In [10]:
work.create_train_task(1, 1, 1)

🆕 새로운 ID 생성: 0


TaskRecord(id=0, name='train', dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2/train___id_0'), config_manager=TaskConfigManager(dir_path=WindowsPath('W:/RM/resources/works/test_2___id_2/train___id_0'), memo_factory=<rm.memo.factory.MemoFactory object at 0x000001B8E24E6590>, CONFIG_NAME='task_config'))

In [11]:
work.task_view.table

,id,0
0,0,train


In [2]:
from resource_manager_mmdetection.env_config import project_root

project_root

PosixPath('/home/submodules/mmdetection')

In [39]:
from dataclasses import dataclass, asdict, is_dataclass, fields
import json
from typing import Any, Type

# ✅ 1. 저장: dataclass → JSON
def save_dataclass_to_json(obj: Any, filepath: str):
    if not is_dataclass(obj):
        raise TypeError("Input must be a dataclass instance.")
    with open(filepath, 'w') as f:
        json.dump(asdict(obj), f, indent=2)

# ✅ 2. 로드: JSON → dataclass (재귀 지원)
def from_dict(cls: Type[Any], data: dict) -> Any:
    """
    재귀적으로 dict 데이터를 dataclass로 변환
    """
    if not is_dataclass(cls):
        raise TypeError("cls must be a dataclass type.")
    kwargs = {}
    for field in fields(cls):
        value = data.get(field.name)
        if is_dataclass(field.type) and isinstance(value, dict):
            value = from_dict(field.type, value)
        kwargs[field.name] = value
    return cls(**kwargs)

def load_dataclass_from_json(cls: Type[Any], filepath: str) -> Any:
    with open(filepath, 'r') as f:
        data = json.load(f)
    return from_dict(cls, data)


In [3]:
import os
os.chdir("W:/RM/src")

In [18]:
from rm.memo import MemoFactory

import json
from pathlib import Path
from pydantic import BaseModel, Field, PrivateAttr
from typing import Type, TypeVar
from rm.memo import FileMemo

T = TypeVar("T", bound="AutoSavingModel")


class AutoSavingModel(BaseModel):
    _memo: FileMemo = PrivateAttr()
    _suspend_sync: bool = PrivateAttr(default=False)

    def __init__(self, **data):
        memo = data.pop("_memo")
        super().__init__(**data)
        self._memo = memo
        self._save()

    def __setattr__(self, name, value):
        super().__setattr__(name, value)
        if not name.startswith("_") and not self._suspend_sync:
            self._save()

    # @property
    # def memo(self):
    #     return self._memo_factory.make_file_memo(Path("test.json"))

    def _save(self):
        self._memo.set(self.model_dump())

    @classmethod
    def load(cls: Type[T], memo: FileMemo) -> T:
        content = memo.get()
        return cls(_memo=memo, **content)

class ModelConfig(AutoSavingModel):
    name: str = Field(default="resnet")
    layers: int = Field(default=50)


file_path = Path("test.json")
memo = MemoFactory().make_file_memo(file_path)

# x = ModelConfig(_memo=memo, name="resnet")
# x._save()

# x.name = "hello1"




In [20]:
memo.content

{}

In [21]:
ModelConfig.load(memo)

ModelConfig(name='resnet', layers=50)

In [5]:
x.memo.file_path

WindowsPath('test.yaml')

In [13]:
class User(BaseModel):
    name: str = Field(..., alias='fullName')
    age: int

u = User(fullName='Alice', age=30)  # ✅ 작동할 수도 있지만...

# 보다 확실하게 작동하게 하려면:
u = User.model_validate({'fullName': 'Alice', 'age': 30})
u

User(name='Alice', age=30)

In [24]:
is_directly_dataclass(AppConfig)

True

In [15]:


# 예시 객체
config = AppConfig(model=ModelConfig(name="resnet", layers=50), debug=True)

# 저장
save_dataclass_to_json(config, 'config.json')

# 로드
loaded_config: AppConfig = load_dataclass_from_json(AppConfig, 'config.json')

print(loaded_config)

loaded_config.model.layers

TypeError: DATA_CLASS.__init__() got an unexpected keyword argument 'model'

In [10]:
loaded_config = load_dataclass_from_json(AppConfig, 'config.json')

print(loaded_config)


AppConfig(model=ModelConfig(name='resnet', layers=50), debug=True)


50